In [46]:
# import packages
from deabook import DEA
from deabook.constant import CET_ADDI, RTS_VRS1, RTS_CRS, OPT_LOCAL

import pandas as pd
# import all data (including the contextual varibale)

kind = "province" # "province" or "country"

if kind == "province":
    dmuname = '省份'
    data = pd.read_excel(r"../../data\china data.xlsx").query('year>2015').reset_index(drop=True)
    region = data[["省份","group"]]
else:
    dmuname = '国家'
    data = pd.read_excel(r"../../data\oecd data.xlsx").query('year>2015').reset_index(drop=True)


yearid = data[['year',dmuname]]
yearid



,year,省份
0,2016,上海市
1,2017,上海市
2,2018,上海市
3,2019,上海市
4,2016,云南省
...,...,...
115,2019,青海省
116,2016,黑龙江省
117,2017,黑龙江省
118,2018,黑龙江省


In [47]:
data

,省份,year,Y,E,L,K,CO2,group
0,上海市,2016,902.794998,11286.777262,1365.24,4.085533e+06,188.140000,E
1,上海市,2017,929.498785,11423.298889,1372.65,4.238459e+06,190.480000,E
2,上海市,2018,951.911601,11072.057032,1375.66,5.790083e+06,190.641610,E
3,上海市,2019,947.300532,11536.380416,1376.20,6.373635e+06,192.912182,E
4,云南省,2016,427.550554,9360.041199,2855.00,4.787612e+06,180.050000,W
...,...,...,...,...,...,...,...,...
115,青海省,2019,62.322152,3566.776509,330.20,1.164860e+06,51.752403,W
116,黑龙江省,2016,394.193978,9307.525536,1776.00,4.500560e+06,269.340000,C
117,黑龙江省,2017,384.949317,9192.427176,1699.00,4.867986e+06,268.660000,C
118,黑龙江省,2018,384.333468,7514.188572,1635.00,4.611428e+06,248.035933,C


In [48]:
yearid[dmuname].unique()

array(['上海市', '云南省', '内蒙古自治区', '北京市', '吉林省', '四川省', '天津市', '宁夏回族自治区',
       '安徽省', '山东省', '山西省', '广东省', '广西壮族自治区', '新疆维吾尔自治区', '江苏省', '江西省',
       '河北省', '河南省', '浙江省', '海南省', '湖北省', '湖南省', '甘肃省', '福建省', '贵州省',
       '辽宁省', '重庆市', '陕西省', '青海省', '黑龙江省'], dtype=object)

In [49]:
data[['Y','K','L','E','CO2']].describe().T

,count,mean,std,min,25%,50%,75%,max
Y,120.0,6.831615e+02,5.283917e+02,5.829009e+01,3.097082e+02,4.859612e+02,8.658057e+02,2.341710e+03
K,120.0,7.740876e+06,4.529324e+06,1.067264e+06,4.628847e+06,6.539850e+06,9.449080e+06,2.098236e+07
L,120.0,2.603739e+03,1.658915e+03,3.242800e+02,1.336750e+03,2.176200e+03,3.746685e+03,6.995000e+03
E,120.0,1.358919e+04,8.131022e+03,1.957435e+03,7.458323e+03,1.135504e+04,1.708449e+04,3.479616e+04
CO2,120.0,3.406669e+02,2.205048e+02,3.985000e+01,1.906012e+02,2.727572e+02,4.577052e+02,9.371169e+02


In [50]:
# data[['Y','K','L','E','CO2']].describe().T.to_markdown("table4.1.md")

In [51]:
ssss

NameError: name 'ssss' is not defined

## DDF

In [52]:
from deabook import DEA
from deabook.constant import CET_ADDI, RTS_VRS1, RTS_CRS, OPT_DEFAULT, OPT_LOCAL

In [53]:
def DDF_VRS():

    model = DEA.DDF2(data,sent = "K L E=Y",  gy=[0], gx=[0,0,1],rts=RTS_VRS1, baseindex=None,refindex=None)
    res = model.optimize(solver="mosek")
    # print(model.info(1))
    return res

DDF_VRS=DDF_VRS()
DDF_VRS

Optimizing locally.


,optimization_status,rho,tei
0,ok,0.000000,1.000000
1,ok,0.000000,1.000000
2,ok,0.000000,1.000000
3,ok,0.079021,0.920979
4,ok,0.606090,0.393910
...,...,...,...
115,ok,0.040751,0.959249
116,ok,0.623318,0.376682
117,ok,0.624060,0.375940
118,ok,0.540542,0.459458


In [54]:
DDF_VRS['EO_VRS'] = DDF_VRS['tei']

DDF_VRS = DDF_VRS.drop(columns = ['optimization_status','rho','tei'])



DDF_VRS2 = pd.concat([DDF_VRS,yearid],axis=1)


In [55]:
DDF_VRS3 = DDF_VRS2.pivot_table(
    values=['EO_VRS',],
    index=dmuname,
    columns='year',
    aggfunc='mean',
    fill_value=None,
    margins=False,
    dropna=True,
    margins_name='All',
    observed=False,
)  

In [56]:
DDF_VRS3.to_markdown("DDF_VRS.md")


In [57]:
SSS

NameError: name 'SSS' is not defined

## figure

In [58]:
ddf_mean_year = DDF_VRS3['EO_VRS'].mean()

xtick2 = tuple(ddf_mean_year.index)

In [59]:
ddf_mean_year.mean()

np.float64(0.5286438776110974)

In [60]:
ddf_mean_year

year
2016    0.525358
2017    0.534393
2018    0.534210
2019    0.520614
dtype: float64

In [61]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib notebook
# plt.figure(figsize=(1,1))
params = {
    'figure.figsize': '8, 4'
}
# plt.rcParams.update(params)
# plt.rc('font',family='Times New Roman')
from matplotlib import rcParams
rcParams['font.family'] = 'SimHei'


import numpy as np

In [62]:
fig, ax = plt.subplots() # 创建图形网格


ax.spines['top'].set_visible(False) # 使上边框不可见
ax.spines['right'].set_visible(False)
# ax2.spines['right'].set_visible(False)

################################ 绘图 #################################

n_groups = 4 # 样本的数目（ref，mut1-3）
index = np.arange(n_groups) # 转换为列表
bar_width = 0.5 # 设置柱子的宽度
# opacity = 0.8 # 设置柱子的透明度
error_config = {'ecolor': 'black','capsize' :2} # 设置误差线为黑色，宽度为2



# 画第一个样本的柱子
rects1 = ax.plot(index, ddf_mean_year,
                color='darkblue',alpha = 0.8,linestyle = "dashed",marker = "s",label='DDF_VRS'
            #      error_kw=error_config,
            #   edgecolor = 'white'
              )

###########################################333

# ax.set_xlabel('Hours in 4 ℃')
ax.set_ylabel( r'$\mathregular{能源效率}$')
#  r'$Difficulty of reducing emissions \gamma$' 
# ax.set_title('Creat by python')
ax.set_xticks(index )
ax.set_xticklabels(xtick2)
# ax2.set_xlabel('Relative gene expression')
# ax.spines["bottom"].set_position(("data",0))
plt.legend()#borderaxespad=10,loc='upper right', fontsize='x-large'mode='expand'

plt.xticks(rotation=80)    # x 轴刻度添加80度旋转
# ax.legend(frameon=False,loc='best',ncol = 1)
if kind=="province":
   plt.savefig("chap41分年份能源效率-省.png", dpi=900, bbox_inches = 'tight')
else:
   plt.savefig("chap41分年份能源效率-国.png", dpi=900, bbox_inches = 'tight')
plt.show()


<IPython.core.display.Javascript object>

In [63]:
ddf_mean_cou = DDF_VRS3['EO_VRS'].mean(axis=1) .sort_values()


In [64]:

ddf_mean_cou

省份
内蒙古自治区      0.163240
新疆维吾尔自治区    0.214346
河北省         0.243310
山西省         0.254045
辽宁省         0.260057
贵州省         0.319397
四川省         0.347403
云南省         0.373178
湖北省         0.377805
甘肃省         0.382193
陕西省         0.395205
湖南省         0.398789
安徽省         0.399029
黑龙江省        0.407162
广西壮族自治区     0.410948
江西省         0.418969
天津市         0.421552
重庆市         0.451696
河南省         0.454317
福建省         0.469928
吉林省         0.505941
山东省         0.575586
浙江省         0.796737
江苏省         0.941189
海南省         0.956602
宁夏回族自治区     0.969549
青海省         0.978342
上海市         0.980245
广东省         0.992556
北京市         1.000000
dtype: float64

In [65]:

xtick2 = tuple(ddf_mean_cou.index)
xtick2

('内蒙古自治区',
 '新疆维吾尔自治区',
 '河北省',
 '山西省',
 '辽宁省',
 '贵州省',
 '四川省',
 '云南省',
 '湖北省',
 '甘肃省',
 '陕西省',
 '湖南省',
 '安徽省',
 '黑龙江省',
 '广西壮族自治区',
 '江西省',
 '天津市',
 '重庆市',
 '河南省',
 '福建省',
 '吉林省',
 '山东省',
 '浙江省',
 '江苏省',
 '海南省',
 '宁夏回族自治区',
 '青海省',
 '上海市',
 '广东省',
 '北京市')

In [66]:
fig, ax = plt.subplots() # 创建图形网格


ax.spines['top'].set_visible(False) # 使上边框不可见
ax.spines['right'].set_visible(False)
# ax2.spines['right'].set_visible(False)

################################ 绘图 #################################
if kind=="province":
   n_groups = 30 # 样本的数目（ref，mut1-3）
else:
   n_groups = 33  

index = np.arange(n_groups) # 转换为列表
bar_width = 0.5 # 设置柱子的宽度
# opacity = 0.8 # 设置柱子的透明度
error_config = {'ecolor': 'black','capsize' :2} # 设置误差线为黑色，宽度为2



# 画第一个样本的柱子

rects2 = ax.bar(index , ddf_mean_cou,
                bar_width/2,
                color='olivedrab',alpha = 0.8,
                 error_kw=error_config,
              edgecolor = 'white',label='DDF_VRS'
              )

###########################################333

# ax.set_xlabel('Hours in 4 ℃')
ax.set_ylabel( r'$\mathregular{能源效率}$')
#  r'$Difficulty of reducing emissions \gamma$' 
# ax.set_title('Creat by python')
ax.set_xticks(index )
ax.set_xticklabels(xtick2)
# ax2.set_xlabel('Relative gene expression')
# ax.spines["bottom"].set_position(("data",0))
plt.legend(loc='upper left')#borderaxespad=10,loc='upper right', fontsize='x-large'mode='expand'

plt.xticks(rotation=80)    # x 轴刻度添加80度旋转
# ax.legend(frameon=False,loc='best',ncol = 1)
if kind=="province":
   plt.savefig("chap41分省份能源效率.png", dpi=900, bbox_inches = 'tight')
else:
   plt.savefig("chap41分国家能源效率.png", dpi=900, bbox_inches = 'tight')

plt.show()


<IPython.core.display.Javascript object>